## Loading and Splitting

In [1]:
!pip install -qU langchain langchain-community pypdf


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
from langchain_community.document_loaders import PyPDFLoader

In [3]:
# Bulk Loading PDF
loaders = [
    PyPDFLoader("MachineLearning-Lecture01.pdf"),
    PyPDFLoader("MachineLearning-Lecture01.pdf"),
    PyPDFLoader("MachineLearning-Lecture02.pdf"),    
    PyPDFLoader("MachineLearning-Lecture03.pdf"),    
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [5]:
# Split
!pip install -qU langchain-text-splitters

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [6]:
splits = text_splitter.split_documents(docs)

In [8]:
len(splits)

208

## Embeddings

In [11]:
!pip install -qU langchain_openai
!pip install -qU python-dotenv


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [13]:
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()

embedding = OpenAIEmbeddings(
    model="text-embedding-3-large"
)

## Vectorstores

In [14]:
!pip install -qU chromadb langchain-chroma


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [21]:
persist_directory = 'docs/chroma1'
!rm -rf ./docs/chroma

In [22]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

In [23]:
print(vectordb._collection.count())

208


## Similarity Search

In [24]:
question = "is there an email i can ask for help"
docs = vectordb.similarity_search(question, k=3)

In [25]:
docs[0].page_content

"cs229-qa@cs.stanford.edu. This goes to an account that's read by all the TAs and me. So \nrather than sending us email individually, if you send email to this account, it will \nactually let us get back to you maximally quickly with answers to your questions.  \nIf you're asking questions about homework problems, please say in the subject line which \nassignment and which question the email refers to, since that will also help us to route \nyour question to the appropriate TA or to me appropriately and get the response back to \nyou quickly.  \nLet's see. Skipping ahead — let's see — for homework, one midterm, one open and term \nproject. Notice on the honor code. So one thing that I think will help you to succeed and \ndo well in this class and even help you to enjoy this class more is if you form a study \ngroup.  \nSo start looking around where you're sitting now or at the end of class today, mingle a \nlittle bit and get to know your classmates. I strongly encourage you to form st

## RAG Agents using "Tools"

In [30]:
from langchain.tools import tool

@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vectordb.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [37]:
!pip install -qU dotenv
!pip install -qU langchain-openai    


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [34]:
load_dotenv()

True

In [38]:
from langchain.chat_models import init_chat_model

model = init_chat_model(
    "openai:gpt-4o",
    temperature=0.5,
    timeout=10,
    max_tokens=1000
)

In [39]:
from langchain.agents import create_agent

tools = [retrieve_context]
# If desired, specify custom instructions
prompt = (
    "You have access to a tool that retrieves context from a blog post. "
    "Use the tool to help answer user queries."
)
agent = create_agent(model, tools, system_prompt=prompt)

In [40]:
query = (
    "What is the content about?\n\n"
    "Who is teaching the course."
)

for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What is the content about?

Who is teaching the course.
================================== Ai Message ==================================
Tool Calls:
  retrieve_context (call_DZMiWaoI6UB3IX3HxdqGkAPT)
 Call ID: call_DZMiWaoI6UB3IX3HxdqGkAPT
  Args:
    query: content of the blog post
  retrieve_context (call_MqXPW6Gw2KOvq4v1nyBXDkux)
 Call ID: call_MqXPW6Gw2KOvq4v1nyBXDkux
  Args:
    query: who is teaching the course
================================= Tool Message =================================
Name: retrieve_context

Source: {'page_label': '1', 'creationdate': '2008-07-11T11:25:23-07:00', 'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'author': '', 'title': '', 'total_pages': 22, 'creator': 'PScript5.dll Version 5.2.2', 'source': 'MachineLearning-Lecture01.pdf', 'moddate': '2008-07-11T11:25:23-07:00', 'page': 0}
Content: MachineLearning-Lecture01  
Instructor (Andrew Ng): Okay. Good morning. Welcome 